In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.graph_objects as go
import os

In [ ]:
df = pd.read_excel("data/consolidated/consolidado_salud.xlsx")

# Asegurarse de que PRECIO_UNITARIO sea numérico
df['PRECIO_UNITARIO'] = pd.to_numeric(df['PRECIO_UNITARIO'], errors='coerce')

# Logaritmo del precio unitario y volatilidad para visualización interna
df['PRECIO_UNITARIO_LOG'] = np.log1p(df['PRECIO_UNITARIO'])



# Conteo de categorías por Fuente (Barplot)
fig1 = px.histogram(df, x='CATEGORIA', color='FUENTE', barmode='group',
                    labels={'CATEGORIA':'Categoría', 'count':'Cantidad de productos', 'FUENTE':'Fuente'})
fig1.update_layout(xaxis_tickangle=-45)
fig1.write_image("results/Salud/cantidad_productos_por_categoria.pdf")
fig1.show()

# 3.2 Precio promedio por categoría y fuente (Heatmap)
precio_categoria_fuente = df.groupby(['CATEGORIA','FUENTE'])['PRECIO_UNITARIO_LOG'].mean().reset_index()
fig2 = px.density_heatmap(precio_categoria_fuente, x='FUENTE', y='CATEGORIA',
                          z='PRECIO_UNITARIO_LOG', color_continuous_scale='Blues',
                          labels={'PRECIO_UNITARIO_LOG':'Precio Unitario', 'CATEGORIA':'Categoría','FUENTE':'Fuente'})
fig2.write_image("results/precio_promedio_por_categoria.pdf")
fig2.show()



# ===============================
# Gráficos de volatilidad
# ===============================

# Calcular volatilidad por producto y fuente
volatilidad_df = df.groupby(['CLAVE_MATCHING','FUENTE']).agg(
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO','std')
).reset_index()

# Logaritmo de precio y volatilidad para visualización interna
volatilidad_df['LOG_PRECIO'] = np.log1p(volatilidad_df['PRECIO_PROMEDIO'])
volatilidad_df['LOG_VOLATILIDAD'] = np.log1p(volatilidad_df['VOLATILIDAD'])

# 4.1 Scatterplot Precio vs Volatilidad
fig3 = px.scatter(volatilidad_df, x='LOG_PRECIO', y='LOG_VOLATILIDAD', color='FUENTE',
                  labels={'LOG_PRECIO':'Precio Unitario', 'LOG_VOLATILIDAD':'Volatilidad', 'FUENTE':'Fuente'})
fig3.write_image("results/precio_vs_volatilidad.pdf")
fig3.show()

# 4.2 Heatmap de volatilidad promedio por categoría y fuente
vol_categoria_fuente = df.groupby(['CATEGORIA','FUENTE'])['PRECIO_UNITARIO'].std().reset_index()
vol_categoria_fuente['VOL_LOG'] = np.log1p(vol_categoria_fuente['PRECIO_UNITARIO'])

fig4 = px.density_heatmap(vol_categoria_fuente, x='FUENTE', y='CATEGORIA', z='VOL_LOG',
                          color_continuous_scale='YlOrRd',
                          labels={'VOL_LOG':'Volatilidad','CATEGORIA':'Categoría','FUENTE':'Fuente'})
fig4.write_image("results/volatilidad_promedio_por_categoria.pdf")
fig4.show()



# ===============================
# Tablas de volatilidad y precio
# ===============================

# Función para graficar tabla y guardar en PDF
def guardar_tabla_pdf(df_tabla, nombre_archivo, titulo=""):
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df_tabla.columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[df_tabla[col] for col in df_tabla.columns],
                   fill_color='lavender',
                   align='left'))
    ])
    fig.update_layout(title_text=titulo)
    fig.write_image(f"results/{nombre_archivo}.pdf")
    fig.show()



# Volatilidad por FUENTE y TIPO_ENVASE
tabla_envase_fuente = df.groupby(['TIPO_ENVASE','FUENTE']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO','std')
).reset_index()


# Volatilidad por FUENTE y CATEGORIA
tabla_categoria_fuente = df.groupby(['CATEGORIA','FUENTE']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO','std')
).reset_index()


# Precio promedio por FUENTE y TIPO_ENVASE
tabla_precio_envase = df.groupby(['TIPO_ENVASE','FUENTE']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    PRECIO_PROMEDIO_LOG=('PRECIO_UNITARIO_LOG','mean')
).reset_index()


# Precio promedio por FUENTE y CATEGORIA
tabla_precio_categoria = df.groupby(['CATEGORIA','FUENTE']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    PRECIO_PROMEDIO_LOG=('PRECIO_UNITARIO_LOG','mean')
).reset_index()

In [ ]:
df = pd.read_excel("data/consolidated/consolidado_salud.xlsx")
df['PRECIO_UNITARIO'] = pd.to_numeric(df['PRECIO_UNITARIO'], errors='coerce')
df['PRECIO_UNITARIO_LOG'] = np.log1p(df['PRECIO_UNITARIO'])

# Calcular volatilidad por producto (general, sin fuente)
vol_df = df.groupby('CLAVE_MATCHING').agg(
    PRECIO_PROMEDIO=('PRECIO_UNITARIO_LOG','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO_LOG','std')
).reset_index()
vol_df['VOLATILIDAD'] = vol_df['VOLATILIDAD'].fillna(0)

# ===============================
# 3. Escalar variables para clustering
# ===============================
scaler = StandardScaler()
X = scaler.fit_transform(vol_df[['PRECIO_PROMEDIO','VOLATILIDAD']])

# ===============================
# 4. Clustering K-Means (solo precio y volatilidad)
# ===============================
k = 3  # Número de clusters
kmeans = KMeans(n_clusters=k, random_state=42)
vol_df['CLUSTER'] = kmeans.fit_predict(X)

# ===============================
# 5. Gráficos de clustering (general)
# ===============================

# Scatterplot Precio vs Volatilidad con clusters
fig1 = px.scatter(
    vol_df,
    x='PRECIO_PROMEDIO', y='VOLATILIDAD',
    color='CLUSTER',
    labels={'PRECIO_PROMEDIO':'Precio Unitario',
            'VOLATILIDAD':'Volatilidad',
            'CLUSTER':'Cluster'}
)
fig1.write_image("results/Salud/clusters_salud_prec_std.pdf")
fig1.show()

# Distribución de clusters (histograma general)
cluster_count = vol_df.groupby('CLUSTER').size().reset_index(name='CANTIDAD')
fig2 = px.bar(cluster_count, x='CLUSTER', y='CANTIDAD',
              labels={'CLUSTER':'Cluster','CANTIDAD':'Cantidad de productos'})
fig2.write_image("results/distribucion_clusters.pdf")
fig2.show()

# ===============================
# 6. Tablas de clusters (solo mostrar)
# ===============================

# Cluster vs Categoría (sin fuente)
tabla_cluster_categoria = df.merge(vol_df[['CLAVE_MATCHING','CLUSTER']], on='CLAVE_MATCHING')
tabla_cluster_categoria = tabla_cluster_categoria.groupby(['CLUSTER','CATEGORIA']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO','std')
).reset_index()
print("Tabla de clusters por Categoría (general):")
display(tabla_cluster_categoria)

# Cluster vs Tipo de envase (sin fuente)
tabla_cluster_envase = df.merge(vol_df[['CLAVE_MATCHING','CLUSTER']], on='CLAVE_MATCHING')
tabla_cluster_envase = tabla_cluster_envase.groupby(['CLUSTER','TIPO_ENVASE']).agg(
    N_PRODUCTOS=('CLAVE_MATCHING','count'),
    PRECIO_PROMEDIO=('PRECIO_UNITARIO','mean'),
    VOLATILIDAD=('PRECIO_UNITARIO','std')
).reset_index()
print("\nTabla de clusters por Tipo de Envase (general):")
display(tabla_cluster_envase)


c:\Users\jhers\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.



Tabla de clusters por Categoría (general):


,CLUSTER,CATEGORIA,N_PRODUCTOS,PRECIO_PROMEDIO,VOLATILIDAD
0,0,ANTICONCEPTIVOS,2172,1.564286,0.693467
1,0,ANTIINFECCIOSOS SISTÉMICOS,11415,1.059985,0.875490
2,0,APARATO CARDIOVASCULAR,5253,1.653594,1.132520
3,0,APARATO DIGESTIVO,3563,1.049046,0.894309
4,0,APARATO GENITOURINARIO Y REPRODUCTOR,176,1.856250,0.549673
5,0,APARATO RESPIRATORIO,1892,1.527415,0.677282
6,0,OFTALMOLÓGICOS,308,1.408084,0.670145
7,0,PIEL Y MUCOSAS,1018,0.992230,0.754524
8,0,"SANGRE, LÍQUIDOS Y ELECTROLITOS",1584,1.112929,0.833013
9,0,SISTEMA NERVIOSO,3374,1.769950,1.062821



Tabla de clusters por Tipo de Envase (general):


,CLUSTER,TIPO_ENVASE,N_PRODUCTOS,PRECIO_PROMEDIO,VOLATILIDAD
0,0,BLISTER,3550,1.716070,0.964445
1,0,BOLSA,44,0.058182,0.005816
2,0,CAJA,14812,1.587144,1.000227
3,0,DISPENSADOR,44,1.310000,0.020231
4,0,FRASCO,10323,0.784220,0.687310
5,0,GRANULADO,44,2.200000,0.101156
6,0,POTE,44,0.196591,0.011801
7,0,SOBRE,132,1.850303,0.571317
8,0,TABS,44,2.179545,0.085125
9,0,TUBO,1562,1.169161,0.675576


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os


df = pd.read_excel("data/consolidated/consolidado_salud.xlsx")
df['PRECIO_UNITARIO'] = pd.to_numeric(df['PRECIO_UNITARIO'], errors='coerce')
df['PRECIO_UNITARIO_LOG'] = np.log1p(df['PRECIO_UNITARIO'])

if not os.path.exists('results'):
    os.makedirs('results')

# Gráfico de caja
fig4 = px.box(
    df, 
    x="CATEGORIA", 
    y="PRECIO_UNITARIO_LOG", 
    color="CATEGORIA", 
    labels={"PRECIO_UNITARIO_LOG": "Precio Unitario (log)", "CATEGORIA": "Categoría"},
    width=900, height=500,
    template="plotly_white"
)

fig4.update_layout(
    showlegend=False,
    margin=dict(l=40, r=40, t=40, b=120)
)

fig4.update_yaxes(range=[0, df['PRECIO_UNITARIO_LOG'].max() + 1])
fig4.update_layout(xaxis={'categoryorder':'total descending'})

fig4.show()
fig4.write_image("results/Salud/boxplot_salud.pdf")

# Gráfico de volatilidad por categoría y fuente
vol_categoria_fuente = df.groupby(['CATEGORIA', 'FUENTE'])['PRECIO_UNITARIO_LOG'].std().reset_index()
vol_categoria_fuente = vol_categoria_fuente.rename(columns={'PRECIO_UNITARIO_LOG': 'VOLATILIDAD'})

fig_vol = go.Figure(data=go.Heatmap(
    z=vol_categoria_fuente['VOLATILIDAD'],
    x=vol_categoria_fuente['FUENTE'],
    y=vol_categoria_fuente['CATEGORIA'],
    colorscale='Teal',  
    colorbar=dict(title='Volatilidad'),
    showscale=True
))

for i, row in vol_categoria_fuente.iterrows():
    fig_vol.add_trace(go.Scatter(
        x=[row['FUENTE']], y=[row['CATEGORIA']],
        text=[f"{row['VOLATILIDAD']:.2f}"],
        mode="text",
        showlegend=False
    ))

fig_vol.update_layout(
    title=None,  
    yaxis_title="Categoría",
    margin=dict(l=40, r=40, t=40, b=120),
    width=900, height=500,
)

fig_vol.show()
fig_vol.write_image("results/Salud/heatmap_volatilidad_salud.pdf")

# Gráfico de línea de evolución del precio
df['FECHA'] = pd.to_datetime(df['FECHA'])
evolucion_precio = df.groupby(['FECHA', 'CATEGORIA'])['PRECIO_UNITARIO_LOG'].mean().reset_index()

fig_line = px.line(
    evolucion_precio, 
    x='FECHA', 
    y='PRECIO_UNITARIO_LOG',
    color='CATEGORIA', 
    markers=True,
    labels={'PRECIO_UNITARIO_LOG': 'Precio Unitario (log)', 'FECHA': 'Fecha', 'CATEGORIA': 'Categoría'},
    width=900, height=500,  
    template="plotly_white"
)

fig_line.update_layout(
    title=None, 
    xaxis_title="Fecha", 
    yaxis_title="Precio Unitario (log)", 
    margin=dict(l=40, r=40, t=40, b=120)
)

fig_line.show()
fig_line.write_image("results/Salud/evolucion_precio_salud.pdf")

# Gráfico de barras para el Coeficiente de Variación (CV)
cv_categoria_fuente = df.groupby(['CATEGORIA', 'FUENTE'])['PRECIO_UNITARIO_LOG'].agg(['mean', 'std']).reset_index()
cv_categoria_fuente['CV'] = cv_categoria_fuente['std'] / cv_categoria_fuente['mean'] * 100
cv_categoria_fuente = cv_categoria_fuente.rename(columns={'mean': 'MEDIA', 'std': 'DESVIACION'})

fig_cv = px.bar(
    cv_categoria_fuente, 
    x='CATEGORIA', 
    y='CV', 
    color='FUENTE', 
    labels={'CV': 'Coeficiente de Variación (%)', 'CATEGORIA': 'Categoría'},
    title=None,
    width=900, height=500,
    template="plotly_white",
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set1
)

fig_cv.update_layout(
    xaxis_title="Categoría", 
    yaxis_title="Coeficiente de Variación (%)", 
    margin=dict(l=40, r=40, t=40, b=120),
    legend_title="Fuente",  
    legend=dict(
        orientation="v",       
        yanchor="top",
        y=0.5,  
        xanchor="left",
        x=1.05  
    )
)

fig_cv.show()
fig_cv.write_image("results/Salud/cof_variacion_salud.pdf")
